In [7]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json

In [8]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [9]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [10]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [11]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = '35.240.149.229:9092'

In [12]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [13]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [14]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [16]:
#Verifier receives a confirmation from “result” topic (message with code “200” with matching vid and txid)
for msg in resultconsumer:
    result = msg.value
    print(result)

{'timestamp': 1679897411, 'vid': 'V226190', 'txid': 'TX01490', 'checksum': 'df4facaf7b61088302d555511687476a', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897421, 'vid': 'V226190', 'txid': 'TX09639', 'checksum': 'b8888db2515a66c93fc2ca038e2601f6', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897427, 'vid': 'V226190', 'txid': 'TX04233', 'checksum': '12ae605b162cea96cb06a9cc729001aa', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897433, 'vid': 'V226190', 'txid': 'TX02470', 'checksum': 'fca9f0bea7e70f52fa5e10d439c23a95', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897438, 'vid': 'V226190', 'txid': 'TX09525', 'checksum': '1234ac9b615826dcdefd4e0e78c6c728', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897444, 'vid': 'V226190', 'txid': 'TX08848', 'checksum': '902d7585f749009704fbbe8e5da08fab', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1679897452, 'vid': 'V226190', 'txid': 'TX04876', 'checksum': '86bbd8f6d9795908b040535b502a6b25', 'code': 200, 

KeyboardInterrupt: 